# Tic-Tac-Toe (Noughts and Crosses)
# Updated Implementation of https://medium.com/@jelle.vankerkvoorde/enhancing-python-games-with-large-language-model-agents-a-noughts-and-crosses-case-study-31e6654b9f45

Even Opus and GPT-4 can't play tic tac toe optimally even when given:
* Game rules or not
* board every time or not
* valid moves or not

### fill .env file with ANTHROPIC_API_KEY=sk_
### for Colab, upload .env to Google Colab in Files

In [1]:
!pip install -q python-dotenv
import os
from dotenv import load_dotenv
load_dotenv()

# print(os.environ['ANTHROPIC_API_KEY'])
!pip install -q anthropic
import anthropic

import time

In [2]:
import random
import re
import ast
import copy

class NoughtsAndCrosses:
    def __init__(self, agent):
        self.agent = agent
        self.n = 3
        self.reset()

    def print_field(self):
        for row in self.board:
            print("|".join(row))
            print("-" * 5)

    def get_board_str(self, board):
        return "\n".join([" ".join(row) for row in board])

    def reset(self):
        self.board = [[str(i + j * self.n + 1) for i in range(self.n)] for j in range(self.n)]
        self.current_player = "X"

    def is_winner(self, player):
        for i in range(self.n):
            if all([cell == player for cell in self.board[i]]) or all([self.board[j][i] == player for j in range(self.n)]):
                return True
        if all([self.board[i][i] == player for i in range(3)]) or all([self.board[i][2 - i] == player for i in range(self.n)]):
            return True
        return False

    def is_board_full(self):
        return all(all(cell in ['X', 'O'] for cell in row) for row in self.board)

    def pos_to_row_col(self, position):
        # Convert the position to row and column
        row = (position - 1) // self.n  # Integer division to find the row
        col = (position - 1) % self.n   # Modulus to find the column
        return row, col

    def make_move(self, position):
        row, col = self.pos_to_row_col(position)
        if self.board[row][col] not in ['X', 'O']:
            self.board[row][col] = self.current_player
            return True
        return False

    def switch_player(self):
        self.current_player = "O" if self.current_player == "X" else "X"

    def start_game(self):
        while True:
            self.print_field()
            if self.current_player == "X":
                position = int(input("Enter your move (1-%s): " % (self.n*self.n)))
                if position < 1 or position > 9:
                    print("Invalid input. Please enter a number between 1 and %s." % (self.n*self.n))
                    continue
            else:
                position = self.agent.make_move(self.board, self.current_player)

            if self.make_move(position):
                if self.is_winner(self.current_player):
                    self.print_field()
                    print(f"Player {self.current_player} wins!")
                    break
                elif self.is_board_full():
                    self.print_field()
                    print("It's a draw!")
                    break
                self.switch_player()
            else:
                print(f"Invalid move {position} by {self.current_player}, try again.")
                break
        self.reset()


class RandomAgent:
    n = 3
    def make_move(self, board, player):
        # LLM logic to determine the move
        # Placeholder implementation; you'll need to integrate with an actual LLM here
        # For now, this just returns a random empty cell
        import random
        empty_cells = [(i, j) for i in range(self.n) for j in range(3) if board[i][j] not in ['X', 'O']]
        return random.choice(empty_cells) if empty_cells else (0, 0)


class LLMAgent:
    n = 3
    game_rules = (
        "I am an AI playing 'Noughts and Crosses' (also known as Tic-Tac-Toe). "
        "The game is played on a %sx%s grid. Each player takes turns placing their symbol (X or O) in an empty cell. "
        "A cell is considered empty if it does not already contain an X or an O. "
        "The first player to align three of their symbols horizontally, vertically, or diagonally wins. "
        "If all cells are filled and no player has aligned three symbols, the game is a draw. "
        "I will play as one of the players. After receiving the current board state, "
        "I will determine my move and respond with just the the position of an empty cell where I want to place my symbol (O), "
        "I must choose a cell that is not already filled with an X or an O."
        "I must choose a value from 1 to %s." % (n, n, n*n)
    )

    def __init__(self):
        # self.model = "claude-3-haiku-20240307"
        self.model = "claude-3-opus-20240229"
        self.max_tokens = 3500
        self.client = anthropic.Anthropic()

    def get_response(self,
                     prompt,
                     system=f'You are an expert tic-tac-toe player who is very focused and thinks exhaustively before making any move.  Here are the game rules: <rules>\n{game_rules}\n</rules>.',
                     model_kwargs={}):
        print(prompt)
        response = self.client.messages.create(
                model=self.model,
                max_tokens=self.max_tokens,
                system=system,
                messages=[
                    {"role": "user", "content": prompt}
                ],
                **model_kwargs,
            )
        return response.content[0].text

    def critique(self,
                     prompt_critique,
                     prompt,
                     position_str,
                     system=f'You are an expert tic-tac-toe judge and critic who is very focused on analyzing the last player move.  Here are the game rules: <rules>\n{game_rules}\n</rules>.',
                     model_kwargs={}):
        response = self.client.messages.create(
                model=self.model,
                max_tokens=self.max_tokens,
                system=system,
                messages=[
                    {"role": "user", "content": prompt},
                    {"role": "assistant", "content": position_str},
                    {"role": "user", "content": prompt_critique}
                ],
                **model_kwargs,
            )
        return response.content[0].text

    def check_move(self, board, position, player):
        game = NoughtsAndCrosses(None)
        game.board = copy.deepcopy(board)
        game.current_player = player
        if game.make_move(position):
            return self.get_board_str(game.board)
        else:
            return f"Invalid move: {position} by {player}"

    def get_board_str(self, board):
        return "\n".join([" ".join(row) for row in board])

    def get_thoughts_and_position(self, response):
        pattern = r'<thinking>(.*?)</thinking>'
        # re.DOTALL allows dot (.) to match newlines as well
        thoughts = re.findall(pattern, response, re.DOTALL)

        pattern = r'<move>(.*?)</move>'
        position = ast.literal_eval(next(iter(re.findall(pattern, response)), '0'))
        return thoughts, position

    def make_move(self, board, player):
        valid_moves = [x for y in board for x in y if x not in ['X', 'O']]
        prompt = f"The current board is:\n{self.get_board_str(board)}\n\nIt's player {player}'s turn. You are player {player}. Valid moves: {valid_moves}.  First provide your thoughts in <thinking> </thinking> xml tags.  Then give your single integer move in <move> </move> xml tags."
        print(prompt)

        response = self.get_response(prompt)
        print(response)
        thoughts, position = self.get_thoughts_and_position(response)
        print(position)

        new_board_str = self.check_move(board, position, player)
        prompt_critique = f"Look at the resulting possible board:\n<possible_board>\n{new_board_str}\n</possible_board>\nSee if you need to change your mind by going back."
        print(prompt_critique)
        response = self.critique(prompt_critique, prompt, f'<move>{position}</move>')
        print(response)
        thoughts, position = self.get_thoughts_and_position(response)
        print(position)

        return position


In [3]:
agent = LLMAgent()

In [4]:
game = NoughtsAndCrosses(agent)

In [5]:
game.start_game()

1|2|3
-----
4|5|6
-----
7|8|9
-----
Enter your move (1-9): 5
1|2|3
-----
4|X|6
-----
7|8|9
-----
The current board is:
1 2 3
4 X 6
7 8 9

It's player O's turn. You are player O. Valid moves: ['1', '2', '3', '4', '6', '7', '8', '9'].  First provide your thoughts in <thinking> </thinking> xml tags.  Then give your single integer move in <move> </move> xml tags.
The current board is:
1 2 3
4 X 6
7 8 9

It's player O's turn. You are player O. Valid moves: ['1', '2', '3', '4', '6', '7', '8', '9'].  First provide your thoughts in <thinking> </thinking> xml tags.  Then give your single integer move in <move> </move> xml tags.
<thinking>
The opponent has played in the center, which is a strong opening move. To counter this, I should aim to play in a corner square. This will give me two potential paths to victory.

If I play in a side square instead, the opponent could easily block my attempts to get three in a row. The corners are more powerful because they control two potential winning paths 